# Practica siete

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
data = pd.read_csv('precio_casas_clasificacion.csv')

data.head()

# Ejercicio 1

In [ ]:
# Apartado 1

X = data.drop('Precio', axis=1) # Eliminamos la columna precio
y = data['Precio'] # Obtenemos la columna precio
X.head() # Mostramos las 5 primeras filas de X
y.head() # Mostramos las 5 primeras filas de y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # Dividimos los datos en train y test

In [ ]:
# Apartado 2

for i in range(1, 21): # Probamos con 20 vecinos
    knn = KNeighborsClassifier(n_neighbors=i) # Creamos el modelo
    knn.fit(X_train, y_train) # Entrenamos el modelo
    y_pred = knn.predict(X_test) # Predecimos los valores
    print(f'K={i:>2} -> Accuracy: {accuracy_score(y_test, y_pred):.5f}') # Mostramos el accuracy

In [ ]:
# Apartado 3

scaler = MinMaxScaler() # Creamos el objeto scaler para escalar los datos entre 0 y 1
X_train_scaled = scaler.fit_transform(X_train) # Escalamos los datos de entrenamiento
X_test_scaled = scaler.transform(X_test) # Escalamos los datos de test

Max_accuracy_apart3 = 0 # Variable para almacenar el accuracy máximo

for i in range(1, 21): # Probamos con 20 vecinos
    knn = KNeighborsClassifier(n_neighbors=i) # Creamos el modelo con el número de vecinos vecinos que queremos probar (i)
    knn.fit(X_train_scaled, y_train) # Entrenamos el modelo
    y_pred = knn.predict(X_test_scaled) # Predecimos los valores
    print(f'K={i:>2} -> Accuracy scaled: {accuracy_score(y_test, y_pred):.5f}') # Mostramos el accuracy
    if accuracy_score(y_test, y_pred) > Max_accuracy_apart3:
        Max_accuracy_apart3 = accuracy_score(y_test, y_pred)

Como se puede comprobar la tasa de acierto cuando está escalado es bastante mayor que cuando no lo está, ya que así nos aseguramos de que todo esté en la misma orden de magnitud por lo que todas las variables tienen la misma importancia independientemente de su escala.

In [ ]:
# Apartado 4
# Calcular tiempo de ejecución del apartado 3 para ball_tree, kd_tree y brute y printearlo

import time

inicio = time.time() # Guardamos el tiempo de inicio de la ejecución

for j in range(1, 5): # Probamos con 4 iteraciones
    for i in range(1, 21):
        knn = KNeighborsClassifier(n_neighbors=i, algorithm='ball_tree') 
        knn.fit(X_train_scaled, y_train)
        y_pred = knn.predict(X_test_scaled)

fin = time.time() # Guardamos el tiempo de fin de la ejecución
print(f'Tiempo de ejecución ball tree: {fin - inicio:.5f}') # Mostramos el tiempo de ejecución

inicio = time.time()

for j in range(1, 5):
    for i in range(1, 21):
        knn = KNeighborsClassifier(n_neighbors=i, algorithm='kd_tree')
        knn.fit(X_train_scaled, y_train)
        y_pred = knn.predict(X_test_scaled)

fin = time.time()
print(f'Tiempo de ejecución kd tree: {fin - inicio:.5f}')

inicio = time.time()

for j in range(1, 5):
    for i in range(1, 21):
        knn = KNeighborsClassifier(n_neighbors=i, algorithm='brute')
        knn.fit(X_train_scaled, y_train)
        y_pred = knn.predict(X_test_scaled)

fin = time.time()
print(f'Tiempo de ejecución brute: {fin - inicio:.5f}')

El que más tarda es el de ball tree con 50 segundos aprox y el que menos el de kd tree con 18 segundos aprox. El brute tarda 25 segundos aprox, menos que el ball tree pero más que el kd tree.

In [ ]:
# Apartado 5

X_train_5, X_val, y_train_5, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42) # Dividimos los datos de entrenamiento en train y validación (80% train y 20% validación)

print("------------ KNN con pesos uniformes --------------")

Max_accuracy_uniform = 0 # Variable para almacenar el accuracy máximo con pesos uniformes
Max_accuracy_k_uniform = 0 # Variable para almacenar el k con el que se obtiene el accuracy máximo con pesos uniformes

for i in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=i, weights='uniform') # Probamos con pesos uniformes
    knn.fit(X_train_5, y_train_5)
    y_pred = knn.predict(X_val)
    print(f'K={i:>2} -> Accuracy scaled weights = uniform: {accuracy_score(y_val, y_pred):.5f}')
    if accuracy_score(y_val, y_pred) > Max_accuracy_uniform: # Si el accuracy es mayor que el máximo actual, lo guardamos
        Max_accuracy_uniform = accuracy_score(y_val, y_pred) # Guardamos el accuracy máximo
        Max_accuracy_k_uniform = i # Guardamos el k con el que se obtiene el accuracy máximo

print("------------ KNN con pesos inversamente proporcionales a la distancia --------------")

Max_accuracy_distance = 0 # Variable para almacenar el accuracy máximo con pesos inversamente proporcionales a la distancia
Max_accuracy_k_distance = 0 # Variable para almacenar el k con el que se obtiene el accuracy máximo con pesos inversamente proporcionales a la distancia

for i in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=i, weights='distance') # Probamos con pesos inversamente proporcionales a la distancia (más cercanos tienen más peso)
    knn.fit(X_train_5, y_train_5)
    y_pred = knn.predict(X_val)
    print(f'K={i:>2} -> Accuracy scaled weights = distance: {accuracy_score(y_val, y_pred):.5f}')
    if accuracy_score(y_val, y_pred) > Max_accuracy_distance:
        Max_accuracy_distance = accuracy_score(y_val, y_pred)
        Max_accuracy_k_distance = i

print(f'El mejor accuracy con pesos uniformes es {Max_accuracy_uniform:.5f} con k = {Max_accuracy_k_uniform}')
print(f'El mejor accuracy con pesos inversamente proporcionales a la distancia es {Max_accuracy_distance:.5f} con k = {Max_accuracy_k_distance}')

if Max_accuracy_uniform > Max_accuracy_distance: # Comparamos los accuracy máximos para ver cuál es el mejor modelo
    best_k = Max_accuracy_k_uniform
    best_weights = 'uniform'
    print(f'El mejor modelo es con pesos uniformes es y k = {best_k}')
else:
    best_k = Max_accuracy_k_distance
    best_weights = 'distance'
    print(f'El mejor modelo es con pesos inversamente proporcionales a la distancia y k = {best_k}')


In [ ]:
knn = KNeighborsClassifier(n_neighbors=best_k, weights=best_weights) # Creamos el modelo con el mejor k y los mejores pesos (uniform o distance, en este caso distance)
knn.fit(X_train_scaled, y_train) # Entrenamos el modelo
y_pred = knn.predict(X_test_scaled) # Predecimos los valores
print(f'Accuracy final: {accuracy_score(y_test, y_pred):.5f}') # Mostramos el accuracy

print(f'Max_accuracy_apart3: {Max_accuracy_apart3:5.f}') # Mostramos el accuracy máximo del apartado 3

# Ejercicio 2